Let's consider creating DCAT-AP compliant datasets from some source data. 
In order to do so we import DCATDataset class first and look into it's annotation:

In [1]:
from sempyro.dcat import DCATDataset
import pprint

dcat_fields = DCATDataset.annotate_model()
pprint.pprint(dcat_fields.fields_description())

{'access_rights': 'Information about who can access the resource or an '
                  'indication of its security status.',
 'conforms_to': 'An established standard to which the described resource '
                'conforms.',
 'contact_point': 'Relevant contact information for the cataloged resource. '
                  'Use of vCard is recommended',
 'creator': 'The entity responsible for producing the resource. Resources of '
            'type foaf:Agent are recommended as values for this property.',
 'description': 'A free-text account of the resource.',
 'distribution': 'An available distribution of the dataset.',
 'first': 'The first resource in an ordered collection or series of resources, '
          'to which the current resource belongs.',
 'frequency': 'The frequency at which a dataset is published.',
 'has_current_version': 'This resource has a more specific, versioned resource '
                        'with equivalent content [PAV].',
 'has_part': 'A related resourc

/Users/anastasianayden/PycharmProjects/pythonProject/pythonProject/HRIPydanticModels/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:355: UserWarning: [typing.Annotated[pydantic_core._pydantic_core.Url, UrlConstraints(max_length=None, allowed_schemes=['http', 'https'], host_required=None, default_host=None, default_port=None, default_path=None)]] is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


Mandatory fields are:

In [2]:
pprint.pprint(dcat_fields.mandatory_fields())

['description', 'title']


Fields datatypes:

In [3]:
pprint.pprint(dcat_fields.get_fields_types())

{'access_rights': {'RDF type': 'uri', 'datatype': 'AccessRights'},
 'conforms_to': {'RDF type': 'uri', 'datatype': 'Url'},
 'contact_point': {'RDF type': 'uri',
                   'datatype': 'List[Union[Url, VCard, Agent]]'},
 'creator': {'RDF type': 'uri', 'datatype': 'List[Union[Url, VCard, Agent]]'},
 'description': {'RDF type': 'literal', 'datatype': 'List[LiteralField]'},
 'distribution': {'RDF type': 'uri', 'datatype': 'List[Url]'},
 'first': {'RDF type': 'uri', 'datatype': 'Url'},
 'frequency': {'RDF type': 'uri', 'datatype': 'Union[Url, Frequency]'},
 'has_current_version': {'RDF type': 'uri', 'datatype': 'Url'},
 'has_part': {'RDF type': 'uri', 'datatype': 'List[Url]'},
 'has_policy': {'RDF type': 'uri', 'datatype': 'ODRLPolicy'},
 'has_version': {'RDF type': 'uri', 'datatype': 'List[Url]'},
 'identifier': {'RDF type': 'rdfs_literal',
                'datatype': 'List[Union[str, LiteralField]]'},
 'in_series': {'RDF type': 'uri', 'datatype': 'List[Url]'},
 'is_referenced_by':

Now when we know something about target let look into the source:

In [4]:
from tabulate import tabulate
import pandas as pd

df = pd.read_csv("./example_data.csv", sep=";")
print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))

+------+-----------------------------+----------------------------------------------------------------------+------------------+------------------------------------------------------+-------------------------------------+
|   id | name                        | description                                                          | author_name      | author_id                                            | keywords                            |
|------+-----------------------------+----------------------------------------------------------------------+------------------+------------------------------------------------------+-------------------------------------|
|    1 | Gryffindor research project | Impact of muggle technical inventions on word's magic presense       | Hermione Granger | https://harrypotter.fandom.com/wiki/Hermione_Granger | magic, technic, muggles             |
|    2 | Slytherin research project  | Comarative analysis of magic powers of muggle-born and blood wizards | Dr

Comparing fields annotation output we see mandatory fields are in place, but some other need to be renamed and preprocessed.

In [5]:
df["keywords"] = df["keywords"].apply(lambda x: x.split(","))
df["id"] = df["id"].apply(lambda x: [str(x)])
df["description"] = df["description"].apply(lambda x: [x])
df["name"] = df["name"].apply(lambda x: [x])
df["creator"] = df.apply(lambda x: [{"full_name": [x["author_name"]], "hasUID": x["author_id"]}], axis=1)
df.drop(columns=["author_name", "author_id"], inplace=True)
df.rename(columns={"id": "identifier", "name": "title", "keywords": "keyword"}, inplace=True)


In [6]:
from rdflib import URIRef

datasets = df.to_dict('records')
dcat_datasets = [DCATDataset(**x) for x in datasets]
for item in dcat_datasets:
    print(item.to_graph(URIRef(item.identifier[0])).serialize())

@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix v: <http://www.w3.org/2006/vcard/ns#> .

<1> a dcat:Dataset ;
    dcterms:creator [ a v:VCard ;
            v:fn "Hermione Granger" ;
            v:hasUID <https://harrypotter.fandom.com/wiki/Hermione_Granger> ] ;
    dcterms:description "Impact of muggle technical inventions on word's magic presense" ;
    dcterms:identifier "1" ;
    dcterms:title "Gryffindor research project" ;
    dcat:keyword " muggles",
        " technic",
        "magic" .


@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix v: <http://www.w3.org/2006/vcard/ns#> .

<2> a dcat:Dataset ;
    dcterms:creator [ a v:VCard ;
            v:fn "Draco Malfoy" ;
            v:hasUID <https://harrypotter.fandom.com/wiki/Draco_Malfoy> ] ;
    dcterms:description "Comarative analysis of magic powers of muggle-born and blood wizards " ;
    dcterms:identifier "2" ;
    dcterms:ti

Say we have a FDP instance where we would like to publish data under a pre-existing catalog with identifier "http://myfdp/catalog/my-catalogue-id-1". Also FDP implementation re-defines general DCATResource.relation as "is_part_of" for a dataset. Thet is why we need to define a child class and extend it with "is_part_of" field. 

In [7]:
from pydantic import AnyUrl, Field
from rdflib import DCTERMS

class FDPDataset(DCATDataset):
    is_part_of: AnyUrl = Field(description="fdp catalog id", rdf_term=DCTERMS.isPartOf, rdf_type="uri")

df["is_part_of"] = "http://myfdp/catalog/my-catalogue-id-1"
fdp_datasets_data = df.to_dict('records')
fdp_datasets = [FDPDataset(**x) for x in fdp_datasets_data]
for item in fdp_datasets:
    print(item.to_graph(URIRef(item.identifier[0])).serialize())


@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix v: <http://www.w3.org/2006/vcard/ns#> .

<1> a dcat:Dataset ;
    dcterms:creator [ a v:VCard ;
            v:fn "Hermione Granger" ;
            v:hasUID <https://harrypotter.fandom.com/wiki/Hermione_Granger> ] ;
    dcterms:description "Impact of muggle technical inventions on word's magic presense" ;
    dcterms:identifier "1" ;
    dcterms:isPartOf <http://myfdp/catalog/my-catalogue-id-1> ;
    dcterms:title "Gryffindor research project" ;
    dcat:keyword " muggles",
        " technic",
        "magic" .


@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix v: <http://www.w3.org/2006/vcard/ns#> .

<2> a dcat:Dataset ;
    dcterms:creator [ a v:VCard ;
            v:fn "Draco Malfoy" ;
            v:hasUID <https://harrypotter.fandom.com/wiki/Draco_Malfoy> ] ;
    dcterms:description "Comarative analysis of magic powers of muggle-born a

Serialized records may be passed to FDP API client